# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 27 2022 3:28PM,249567,20,8698981,"Exact-Rx, Inc.",Released
1,Oct 27 2022 3:28PM,249567,20,8698982,"Exact-Rx, Inc.",Released
2,Oct 27 2022 3:28PM,249567,20,8698983,"Exact-Rx, Inc.",Released
3,Oct 27 2022 3:18PM,249565,10,0086122905,ISDIN Corporation,Released
4,Oct 27 2022 3:18PM,249565,10,0086122908,ISDIN Corporation,Released
5,Oct 27 2022 3:18PM,249565,10,0086122904,ISDIN Corporation,Released
6,Oct 27 2022 3:18PM,249565,10,0086122906,ISDIN Corporation,Released
7,Oct 27 2022 3:18PM,249565,10,0086122907,ISDIN Corporation,Released
8,Oct 27 2022 3:18PM,249565,10,0086122917,ISDIN Corporation,Released
9,Oct 27 2022 3:18PM,249565,10,0086122918,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,249561,Executing,4
25,249564,Released,7
26,249565,Released,16
27,249566,Released,1
28,249567,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249561,NaN,4.0,NaN
249564,NaN,NaN,7.0
249565,NaN,NaN,16.0
249566,NaN,NaN,1.0
249567,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249472,30.0,10.0,1.0
249479,0.0,1.0,0.0
249485,0.0,1.0,0.0
249494,0.0,1.0,0.0
249512,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249472,30,10,1
249479,0,1,0
249485,0,1,0
249494,0,1,0
249512,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249472,30,10,1
1,249479,0,1,0
2,249485,0,1,0
3,249494,0,1,0
4,249512,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249472,30,10,1
1,249479,,1,
2,249485,,1,
3,249494,,1,
4,249512,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 27 2022 3:28PM,249567,20,"Exact-Rx, Inc."
3,Oct 27 2022 3:18PM,249565,10,ISDIN Corporation
19,Oct 27 2022 3:17PM,249564,10,ISDIN Corporation
26,Oct 27 2022 3:16PM,249566,21,"NBTY Global, Inc."
27,Oct 27 2022 3:04PM,249561,10,Bio-PRF
31,Oct 27 2022 2:57PM,249558,10,ISDIN Corporation
44,Oct 27 2022 2:44PM,249555,21,"NBTY Global, Inc."
45,Oct 27 2022 2:39PM,249554,21,"NBTY Global, Inc."
46,Oct 27 2022 2:10PM,249549,21,"NBTY Global, Inc."
47,Oct 27 2022 1:48PM,249544,15,"Person & Covey, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 27 2022 3:28PM,249567,20,"Exact-Rx, Inc.",,,3
1,Oct 27 2022 3:18PM,249565,10,ISDIN Corporation,,,16
2,Oct 27 2022 3:17PM,249564,10,ISDIN Corporation,,,7
3,Oct 27 2022 3:16PM,249566,21,"NBTY Global, Inc.",,,1
4,Oct 27 2022 3:04PM,249561,10,Bio-PRF,,4,
5,Oct 27 2022 2:57PM,249558,10,ISDIN Corporation,,5,8
6,Oct 27 2022 2:44PM,249555,21,"NBTY Global, Inc.",,,1
7,Oct 27 2022 2:39PM,249554,21,"NBTY Global, Inc.",,,1
8,Oct 27 2022 2:10PM,249549,21,"NBTY Global, Inc.",,,1
9,Oct 27 2022 1:48PM,249544,15,"Person & Covey, Inc.",,,36


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 27 2022 3:28PM,249567,20,"Exact-Rx, Inc.",3,,
1,Oct 27 2022 3:18PM,249565,10,ISDIN Corporation,16,,
2,Oct 27 2022 3:17PM,249564,10,ISDIN Corporation,7,,
3,Oct 27 2022 3:16PM,249566,21,"NBTY Global, Inc.",1,,
4,Oct 27 2022 3:04PM,249561,10,Bio-PRF,,4,
5,Oct 27 2022 2:57PM,249558,10,ISDIN Corporation,8,5,
6,Oct 27 2022 2:44PM,249555,21,"NBTY Global, Inc.",1,,
7,Oct 27 2022 2:39PM,249554,21,"NBTY Global, Inc.",1,,
8,Oct 27 2022 2:10PM,249549,21,"NBTY Global, Inc.",1,,
9,Oct 27 2022 1:48PM,249544,15,"Person & Covey, Inc.",36,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 27 2022 3:28PM,249567,20,"Exact-Rx, Inc.",3,,
1,Oct 27 2022 3:18PM,249565,10,ISDIN Corporation,16,,
2,Oct 27 2022 3:17PM,249564,10,ISDIN Corporation,7,,
3,Oct 27 2022 3:16PM,249566,21,"NBTY Global, Inc.",1,,
4,Oct 27 2022 3:04PM,249561,10,Bio-PRF,,4,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 27 2022 3:28PM,249567,20,"Exact-Rx, Inc.",3.0,NaN,NaN
1,Oct 27 2022 3:18PM,249565,10,ISDIN Corporation,16.0,NaN,NaN
2,Oct 27 2022 3:17PM,249564,10,ISDIN Corporation,7.0,NaN,NaN
3,Oct 27 2022 3:16PM,249566,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Oct 27 2022 3:04PM,249561,10,Bio-PRF,NaN,4.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 27 2022 3:28PM,249567,20,"Exact-Rx, Inc.",3.0,0.0,0.0
1,Oct 27 2022 3:18PM,249565,10,ISDIN Corporation,16.0,0.0,0.0
2,Oct 27 2022 3:17PM,249564,10,ISDIN Corporation,7.0,0.0,0.0
3,Oct 27 2022 3:16PM,249566,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Oct 27 2022 3:04PM,249561,10,Bio-PRF,0.0,4.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,998248,31.0,9.0,0.0
15,1497087,97.0,21.0,30.0
16,998039,3.0,1.0,0.0
20,748546,3.0,2.0,0.0
21,1746797,7.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,998248,31.0,9.0,0.0
1,15,1497087,97.0,21.0,30.0
2,16,998039,3.0,1.0,0.0
3,20,748546,3.0,2.0,0.0
4,21,1746797,7.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,31.0,9.0,0.0
1,15,97.0,21.0,30.0
2,16,3.0,1.0,0.0
3,20,3.0,2.0,0.0
4,21,7.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,31.0
1,15,Released,97.0
2,16,Released,3.0
3,20,Released,3.0
4,21,Released,7.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,20,21
Status,,,,,
Completed,0.0,30.0,0.0,0.0,0.0
Executing,9.0,21.0,1.0,2.0,0.0
Released,31.0,97.0,3.0,3.0,7.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,20,21
0,Completed,0.0,30.0,0.0,0.0,0.0
1,Executing,9.0,21.0,1.0,2.0,0.0
2,Released,31.0,97.0,3.0,3.0,7.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,20,21
0,Completed,0.0,30.0,0.0,0.0,0.0
1,Executing,9.0,21.0,1.0,2.0,0.0
2,Released,31.0,97.0,3.0,3.0,7.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()